## Quest
- 업무분장(전처리, 모델학습).
- RecurrenceOfSurgery.csv 사용
- 목표변수 : 범주형, 설명변수 : 최소 6개 
- 서비스 대상과 목표 설명, 변수 선택 이유

### 변수설정
0. 서비스 대상: 환자
    * 수술실패에 영향을 미치는 요인들이 무엇이 있을까?
1. target: 수술실패여부
2. features 
    * 범주형: 고혈압여부
    * 범주형: 성별
    * 범주형: 신부전여부
    * 연속형: 연령
    * 연속형: 체중
    * 연속형: 수술시간

In [1]:
import pandas as pd

In [2]:
df_ROS = pd.read_csv('../../datasets/RecurrenceOfSurgery.csv')
df_ROS[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [3]:
df_ROS['수술시간'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1894 entries, 0 to 1893
Series name: 수술시간
Non-Null Count  Dtype  
--------------  -----  
1840 non-null   float64
dtypes: float64(1)
memory usage: 14.9 KB


#### 전처리

In [4]:
df_ROS_select = df_ROS[['수술실패여부', '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간']]
df_ROS_select[:2]

,수술실패여부,고혈압여부,성별,신부전여부,연령,체중,수술시간
0,0,0,2,0,66,60.3,68.0
1,0,0,1,0,47,71.7,31.0


In [5]:
# null값 확인 -> 수술시간 null값 존재
df_ROS_select.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간      54
dtype: int64

In [6]:
# null값 삭제
df_ROS_select_drop = df_ROS_select.dropna()
df_ROS_select_drop.isnull().sum()

수술실패여부    0
고혈압여부     0
성별        0
신부전여부     0
연령        0
체중        0
수술시간      0
dtype: int64

#### apply()함수를 사용하여 null값 채우기

In [7]:
# 1. target : '수술시간', feature: '성별'
target = df_ROS_select_drop[['수술시간']]
feature = df_ROS_select_drop[['성별']]
target.shape, feature.shape

((1840, 1), (1840, 1))

In [8]:
# 2. null값이 없는 데이터와 실제값을 사용하여 회귀모델 훈련
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(feature, target)

LinearRegression()

In [9]:
model.predict(feature)

array([[62.40798859],
       [61.85601405],
       [61.85601405],
       ...,
       [61.85601405],
       [61.85601405],
       [62.40798859]])

In [16]:
# apply()
def convert_notnull(row) :
    if row == None :
        result = model.predict(row)
        return result
    else :
        result1 = df_ROS_select_drop['수술시간']
        return result1

In [21]:
df_ROS_select['수술시간'].apply(convert_notnull)

,0,1,2,3,4,5,6,7,8,9,...,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893
0,68.0,31.0,78.0,73.0,29.0,47.0,7.0,85.0,49.0,59.0,...,50.0,70.0,30.0,75.0,30.0,80.0,20.0,50.0,25.0,45.0
1,68.0,31.0,78.0,73.0,29.0,47.0,7.0,85.0,49.0,59.0,...,50.0,70.0,30.0,75.0,30.0,80.0,20.0,50.0,25.0,45.0
2,68.0,31.0,78.0,73.0,29.0,47.0,7.0,85.0,49.0,59.0,...,50.0,70.0,30.0,75.0,30.0,80.0,20.0,50.0,25.0,45.0
3,68.0,31.0,78.0,73.0,29.0,47.0,7.0,85.0,49.0,59.0,...,50.0,70.0,30.0,75.0,30.0,80.0,20.0,50.0,25.0,45.0
4,68.0,31.0,78.0,73.0,29.0,47.0,7.0,85.0,49.0,59.0,...,50.0,70.0,30.0,75.0,30.0,80.0,20.0,50.0,25.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,68.0,31.0,78.0,73.0,29.0,47.0,7.0,85.0,49.0,59.0,...,50.0,70.0,30.0,75.0,30.0,80.0,20.0,50.0,25.0,45.0
1890,68.0,31.0,78.0,73.0,29.0,47.0,7.0,85.0,49.0,59.0,...,50.0,70.0,30.0,75.0,30.0,80.0,20.0,50.0,25.0,45.0
1891,68.0,31.0,78.0,73.0,29.0,47.0,7.0,85.0,49.0,59.0,...,50.0,70.0,30.0,75.0,30.0,80.0,20.0,50.0,25.0,45.0
1892,68.0,31.0,78.0,73.0,29.0,47.0,7.0,85.0,49.0,59.0,...,50.0,70.0,30.0,75.0,30.0,80.0,20.0,50.0,25.0,45.0


#### Scaling & Encoding

##### - OneHotEncoding

In [ ]:
# 범주형 데이터 확인 : '고혈압여부', '성별', '신부전여부'
df_ROS_select['고혈압여부'].value_counts(),df_ROS_select['성별'].value_counts(),df_ROS_select['신부전여부'].value_counts()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# 범주형 설명변수 OneHotEncoding
oneHotEncoder = OneHotEncoder() # 인스턴스화
oneHotEncoder.fit(df_ROS_select[['고혈압여부', '성별', '신부전여부']])

In [ ]:
oneHotEncoder.categories_

In [ ]:
encoded_data = oneHotEncoder.transform(df_ROS_select[['고혈압여부', '성별', '신부전여부']]).toarray()
encoded_data.shape

In [ ]:
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['고혈압여부', '성별', '신부전여부']))
df_encoded_data[:2]

In [ ]:
# 병합
df_ROS_select= pd.concat([df_ROS_select.reset_index(drop=True), df_encoded_data.reset_index(drop=True)], axis=1)
df_ROS_select[:2]

In [ ]:
df_ROS_select.shape

##### Scaling

In [ ]:
df_ROS_select.columns

In [ ]:
target = df_ROS_select['수술실패여부']
features = df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부'])

In [ ]:
features.columns

MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
minMaxScaler = MinMaxScaler() #인스턴스화
features= minMaxScaler.fit_transform(features)
features.shape